In [3]:
from flask import Flask, render_template, send_file
import psycopg2
import pandas as pd
import plotly.express as px
import scipy.stats as st
import plotly 
import plotly.utils 
import json
import numpy as np

In [4]:
conn_string = "host='localhost' dbname='phone_usage_db' user='postgres' password='postgres'"
conn = psycopg2.connect(conn_string)

print("Database opened successfully")
cursor = conn.cursor();
query = 'select * from phone_user_behavior'
cursor.execute(query)

colnames = [desc[0] for desc in cursor.description]


print(colnames)


phone_df = pd.DataFrame(cursor.fetchall(), columns=colnames)

conn.close()

print("Database Closed successfully")

Database opened successfully
['user_id', 'device_model', 'operating_system', 'app_usage_minutes_per_day', 'screen_time_hours_per_day', 'battery_drain_mah_per_day', 'number_of_apps_installed', 'data_usage_mb_per_day', 'age', 'gender', 'user_behavior_class']
Database Closed successfully


In [6]:
phone_df

,user_id,device_model,operating_system,app_usage_minutes_per_day,screen_time_hours_per_day,battery_drain_mah_per_day,number_of_apps_installed,data_usage_mb_per_day,age,gender,user_behavior_class
0,1,Google Pixel 5,Android,393,6.4,1872,67,1122,40,Male,4
1,2,OnePlus 9,Android,268,4.7,1331,42,944,47,Female,3
2,3,Xiaomi Mi 11,Android,154,4.0,761,32,322,42,Male,2
3,4,Google Pixel 5,Android,239,4.8,1676,56,871,20,Male,3
4,5,iPhone 12,iOS,187,4.3,1367,58,988,31,Female,3
...,...,...,...,...,...,...,...,...,...,...,...
695,696,iPhone 12,iOS,92,3.9,1082,26,381,22,Male,2
696,697,Xiaomi Mi 11,Android,316,6.8,1965,68,1201,59,Male,4
697,698,Google Pixel 5,Android,99,3.1,942,22,457,50,Female,2
698,699,Samsung Galaxy S21,Android,62,1.7,431,13,224,44,Male,1


In [4]:
json_data = phone_df.to_json(orient='index')

In [13]:
  phone_df_battery = phone_df.copy()
bins = [18, 25, 35, 50]
labels = ["18-25", "25-35", "35-50"]
phone_df_battery['age_groups'] = pd.cut(phone_df_battery['age'], bins, labels=labels)
# Calculating mean battery drain for each age group
mean_battery_age = phone_df_battery.groupby(["age_groups"])["battery_drain_mah_per_day"].mean()
mean_battery_age_df = pd.DataFrame({"age_groups": mean_battery_age.index, "average_battery_drain": mean_battery_age.values})
# Creating a bar chart using Plotly
fig = px.bar(
    mean_battery_age_df,
    x="age_groups",
    y="average_battery_drain",
    title="Average Battery Drain Per Age Group",
    labels={"age_groups": "Age Groups", "average_battery_drain": "Battery Drain (mah/day)"},
    text_auto=True
)
# Customizing the figure's appearance
fig.update_layout(
    xaxis_title="Age Groups",
    yaxis_title="Battery Drain (mah/day)",
    yaxis_range=[0, mean_battery_age.max() + 500],
    template="plotly_white"
)

graph_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

print(graph_json)

{"data": [{"alignmentgroup": "True", "hovertemplate": "Age Groups=%{x}<br>Battery Drain (mah/day)=%{y}<extra></extra>", "legendgroup": "", "marker": {"color": "#636efa", "pattern": {"shape": ""}}, "name": "", "offsetgroup": "", "orientation": "v", "showlegend": false, "textposition": "auto", "texttemplate": "%{y}", "x": ["18-25", "25-35", "35-50"], "xaxis": "x", "y": [1629.1083333333333, 1465.034090909091, 1500.1068376068376], "yaxis": "y", "type": "bar"}], "layout": {"template": {"data": {"barpolar": [{"marker": {"line": {"color": "white", "width": 0.5}, "pattern": {"fillmode": "overlay", "size": 10, "solidity": 0.2}}, "type": "barpolar"}], "bar": [{"error_x": {"color": "#2a3f5f"}, "error_y": {"color": "#2a3f5f"}, "marker": {"line": {"color": "white", "width": 0.5}, "pattern": {"fillmode": "overlay", "size": 10, "solidity": 0.2}}, "type": "bar"}], "carpet": [{"aaxis": {"endlinecolor": "#2a3f5f", "gridcolor": "#C8D4E3", "linecolor": "#C8D4E3", "minorgridcolor": "#C8D4E3", "startlinecol

/var/folders/c5/49gjf4515dq14r1sfkhgdnv00000gn/T/ipykernel_15665/3701107589.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [22]:
screen_time_df = phone_df.copy()
bins = [18, 25, 35, 50]
labels = ["18-25", "25-35", "35-50"]
screen_time_df['age_groups'] = pd.cut(screen_time_df['age'], bins, labels=labels)
grouped_screen_time_df = screen_time_df.groupby(['age_groups'])['screen_time_hours_per_day'].mean()

grouped_screen_time_df = pd.DataFrame({'Age Groups':grouped_screen_time_df.index,'Average Screen On Time (hr/day)':grouped_screen_time_df.values})

fig_screen_time = px.bar(
    grouped_screen_time_df.reset_index(),  # Temporarily reset index for plotting
    x='Age Groups',
    y='Average Screen On Time (hr/day)',

    barmode='group',
    labels={'average_screen_time': 'Average Screen On Time (hr/day)', 'age_groups': 'Age Groups'}
)

fig.show()

/var/folders/c5/49gjf4515dq14r1sfkhgdnv00000gn/T/ipykernel_30454/1087776083.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [13]:
data_usage_df = phone_df.copy()

bins = [18, 25, 35, 50]
labels = ["18-25", "25-35", "35-50"]
data_usage_df['age_groups'] = pd.cut(data_usage_df['age'], bins, labels=labels)

grouped_data_usage = data_usage_df.groupby(['age_groups'])['data_usage_mb_per_day'].mean()

grouped_data_usage_df = pd.DataFrame({'age_groups':grouped_data_usage.index,'avarage_data_usage':grouped_data_usage.values})

fig = px.bar(
    grouped_data_usage_df,
    x='age_groups',
    y="avarage_data_usage",
    labels={'age_groups': "Age Groups","average_data_usage":'Average Data Usage (mb)'}
)

fig.update_layout(
    yaxis_range=[0, grouped_data_usage.max() + 500]
)





/var/folders/c5/49gjf4515dq14r1sfkhgdnv00000gn/T/ipykernel_30454/1789122245.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [53]:
os_types_df = phone_df.copy()
bins = [18, 25, 35, 50]
labels = ["18-25", "25-35", "35-50"]
os_types_df['age_groups'] = pd.cut(os_types_df['age'], bins, labels=labels)

grouped_os_types_df = os_types_df.groupby(['age_groups', 'operating_system'])['user_id'].count().to_frame('user_count')

# Rename columns in place without assignment
grouped_os_types_df.index.set_names(['Age Groups', 'Operating System'], inplace=True)



fig = px.bar(
    grouped_os_types_df.reset_index(),  # Temporarily reset index for plotting
    x='Age Groups',
    y='user_count',
    color='Operating System',
    barmode='group',
    title='Number of Users by Age Group and Operating System',
    labels={'user_count': 'Number of Users', 'age_groups': 'Age Groups'}
)





/var/folders/c5/49gjf4515dq14r1sfkhgdnv00000gn/T/ipykernel_21853/2509833108.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

